# Data preprocessing

Author : Ivona 


* Feature Sampling
* Dimensionality Reduction
* Data Scaling
* Feature Encoding
* Removing unnecessary data 


# 

# Homework 3 same as the first but this time include Ticket as well to achive at least >=0.87 accuracy Vasil Stamenkoski


We removed Ticket from our features assuming that it's identification variable
However, it appears that tickets are also predicors, multiple passengers  had the same ticket number

Task 0: 
 * Create a classifier model for target Survived as the flow is. Try SVM and Decision Tree.

Task 1:  
 *  Try to Label Encode/Ordinal/One-hot the **Ticket** feature and try it on both SVM and Decision Tree classifier
    We should use label encoding since we have many possible values
 *  You should get better performance on both models.  
    For Decision Tree the accuracy score (>= of 0.87) on test is achievable.   
    If that's not the case try to tune the parameters


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns    
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.preprocessing import MaxAbsScaler, StandardScaler
from sklearn.metrics import roc_curve, confusion_matrix, classification_report
from collections import Counter
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import string
from sklearn.preprocessing import LabelEncoder

In [2]:
# read the dataset

df = pd.read_csv('titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [4]:
df = df.drop(['PassengerId','Cabin','Name'], axis=1)

In [5]:
df.dtypes

Survived      int64
Pclass        int64
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Embarked     object
dtype: object

In [6]:
train_df, test_df = train_test_split(df,test_size=0.2)

In [7]:
print('Train dataset shape is: ',train_df.shape)
print('Test dataset shape is: ',test_df.shape)

Train dataset shape is:  (712, 9)
Test dataset shape is:  (179, 9)


In [8]:
print(train_df.describe())
print(test_df.describe())

         Survived      Pclass         Age       SibSp       Parch       Fare
count  712.000000  712.000000  584.000000  712.000000  712.000000  712.00000
mean     0.389045    2.301966   29.585616    0.519663    0.373596   32.75893
std      0.487876    0.842003   14.416291    1.090636    0.774618   51.70354
min      0.000000    1.000000    0.750000    0.000000    0.000000    0.00000
25%      0.000000    2.000000   20.000000    0.000000    0.000000    7.92500
50%      0.000000    3.000000   28.000000    0.000000    0.000000   14.45625
75%      1.000000    3.000000   38.250000    1.000000    0.000000   31.27500
max      1.000000    3.000000   80.000000    8.000000    5.000000  512.32920
         Survived      Pclass        Age       SibSp       Parch        Fare
count  179.000000  179.000000  130.00000  179.000000  179.000000  179.000000
mean     0.363128    2.335196   30.20900    0.536313    0.413408   29.997716
std      0.482250    0.813814   15.05822    1.152706    0.922283   40.780530

In [9]:
print(train_df.describe())
print(test_df.describe())

         Survived      Pclass         Age       SibSp       Parch       Fare
count  712.000000  712.000000  584.000000  712.000000  712.000000  712.00000
mean     0.389045    2.301966   29.585616    0.519663    0.373596   32.75893
std      0.487876    0.842003   14.416291    1.090636    0.774618   51.70354
min      0.000000    1.000000    0.750000    0.000000    0.000000    0.00000
25%      0.000000    2.000000   20.000000    0.000000    0.000000    7.92500
50%      0.000000    3.000000   28.000000    0.000000    0.000000   14.45625
75%      1.000000    3.000000   38.250000    1.000000    0.000000   31.27500
max      1.000000    3.000000   80.000000    8.000000    5.000000  512.32920
         Survived      Pclass        Age       SibSp       Parch        Fare
count  179.000000  179.000000  130.00000  179.000000  179.000000  179.000000
mean     0.363128    2.335196   30.20900    0.536313    0.413408   29.997716
std      0.482250    0.813814   15.05822    1.152706    0.922283   40.780530

In [10]:
for column in train_df.columns:
    print(column, len(train_df[column].unique()))

Survived 2
Pclass 3
Sex 2
Age 85
SibSp 7
Parch 6
Ticket 571
Fare 225
Embarked 4


In [11]:
train_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
483,1,3,female,63.0,0,0,4134,9.5875,S
514,0,3,male,24.0,0,0,349209,7.4958,S
870,0,3,male,26.0,0,0,349248,7.8958,S
662,0,1,male,47.0,0,0,5727,25.5875,S
16,0,3,male,2.0,4,1,382652,29.1250,Q
...,...,...,...,...,...,...,...,...,...
42,0,3,male,NaN,0,0,349253,7.8958,C
617,0,3,female,26.0,1,0,A/5. 3336,16.1000,S
571,1,1,female,53.0,2,0,11769,51.4792,S
599,1,1,male,49.0,1,0,PC 17485,56.9292,C


In [12]:
print(train_df.isna().any())
print(test_df.isna().any())

Survived    False
Pclass      False
Sex         False
Age          True
SibSp       False
Parch       False
Ticket      False
Fare        False
Embarked     True
dtype: bool
Survived    False
Pclass      False
Sex         False
Age          True
SibSp       False
Parch       False
Ticket      False
Fare        False
Embarked     True
dtype: bool


In [13]:
print(train_df['Embarked'].unique())

['S' 'Q' 'C' nan]


In [14]:
train_df['Embarked'] = train_df['Embarked'].fillna(train_df['Embarked'].value_counts().index[0])

In [15]:
#Check if worked
print(train_df['Embarked'].unique())

['S' 'Q' 'C']


In [16]:
train_df.Age.describe()

count    584.000000
mean      29.585616
std       14.416291
min        0.750000
25%       20.000000
50%       28.000000
75%       38.250000
max       80.000000
Name: Age, dtype: float64

In [17]:
zero_not_accepted = ['Age']
for column in zero_not_accepted:
    train_df[column] = train_df[column].replace(0,np.NaN)
    mean = int(train_df[column].mean(skipna=True))
    train_df[column] = train_df[column].replace(np.NaN,mean)

In [18]:
##Chekc if there is still missing data
train_df[['Pclass', 'Sex','Age','SibSp','Parch','Ticket','Fare','Embarked']].isna().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Embarked    0
dtype: int64

In [19]:
## Trying to take of ticket
train_df.Ticket[:10]

C:\Users\vasko\AppData\Local\Temp\ipykernel_18984\358783052.py:2: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  train_df.Ticket[:10]


483      4134
514    349209
870    349248
662      5727
16     382652
712     19996
766    112379
19       2649
826      1601
312    250651
Name: Ticket, dtype: object

In [20]:
train_df.Ticket.describe()

count         712
unique        571
top       3101295
freq            6
Name: Ticket, dtype: object

In [21]:
#create function that takes ticket feature and returns list of ticket_types
def ticket_sep(data_ticket):
    ticket_type = []

    for i in range(len(data_ticket)):

            ticket =data_ticket.iloc[i]

            for c in string.punctuation:
                ticket = ticket.replace(c,"")
                splited_ticket = ticket.split(" ")   
            if len(splited_ticket) == 1:
                ticket_type.append('NO')
            else: 
                ticket_type.append(splited_ticket[0])
    return ticket_type 

In [22]:
train_df["ticket_type"] = ticket_sep(train_df.Ticket)

train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,ticket_type
483,1,3,female,63.0,0,0,4134,9.5875,S,NO
514,0,3,male,24.0,0,0,349209,7.4958,S,NO
870,0,3,male,26.0,0,0,349248,7.8958,S,NO
662,0,1,male,47.0,0,0,5727,25.5875,S,NO
16,0,3,male,2.0,4,1,382652,29.1250,Q,NO


In [23]:
test_df["ticket_type"]= ticket_sep(test_df.Ticket)

test_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,ticket_type
154,0,3,male,NaN,0,0,Fa 265302,7.3125,S,Fa
56,1,2,female,21.0,0,0,C.A. 31026,10.5000,S,CA
697,1,3,female,NaN,0,0,35852,7.7333,Q,NO
52,1,1,female,49.0,1,0,PC 17572,76.7292,C,PC
601,0,3,male,NaN,0,0,349214,7.8958,S,NO


In [24]:
# check how many ticket types are there for each ticket type and visualize:
print(train_df["ticket_type"].value_counts())

NO         534
PC          51
CA          28
A5          19
SOTONOQ     11
STONO       10
WC           9
A4           7
STONO2       5
SCPARIS      5
SOC          4
C            4
SCParis      4
FCC          3
SCAH         3
SOPP         3
PPP          2
PP           2
SCA4         1
SP           1
SWPP         1
SC           1
WEP          1
SOTONO2      1
AS           1
SOP          1
Name: ticket_type, dtype: int64


In [25]:
# for those types that have less than 15 samples in training set, assign type to 'OTHER':


for t in train_df['ticket_type'].unique():
    if len([train_df['ticket_type']==t]) > 16:
        train_df.loc[train_df.ticket_type ==t, 'ticket_type'] = 'OTHER_T'
       
    
for t in test_df['ticket_type'].unique():
    if t not in train_df['ticket_type'].unique():
        test_df.loc[test_df.ticket_type ==t, 'ticket_type'] = 'OTHER_T'
        
print(train_df['ticket_type'].unique())
print(test_df['ticket_type'].unique())

['NO' 'SOC' 'SOTONOQ' 'A5' 'PC' 'CA' 'A4' 'WC' 'SOPP' 'SCA4' 'STONO' 'FCC'
 'SP' 'SCPARIS' 'SCAH' 'SWPP' 'STONO2' 'C' 'SCParis' 'PPP' 'SC' 'PP' 'WEP'
 'SOTONO2' 'AS' 'SOP']
['OTHER_T' 'CA' 'NO' 'PC' 'FCC' 'SCPARIS' 'PP' 'WEP' 'WC' 'C' 'SOTONOQ'
 'SOTONO2' 'STONO' 'SOC' 'A5' 'STONO2' 'SWPP']


In [26]:
# where ticket_type is 'SOTONOQ' convert it to 'A5'
train_df["ticket_type"] = np.where(train_df["ticket_type"]=='SOTONOQ', 'A5', train_df["ticket_type"])

In [27]:
##Drop Ticket
train_df = train_df.drop(['Ticket'], axis=1)

In [28]:
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,ticket_type
483,1,3,female,63.0,0,0,9.5875,S,NO
514,0,3,male,24.0,0,0,7.4958,S,NO
870,0,3,male,26.0,0,0,7.8958,S,NO
662,0,1,male,47.0,0,0,25.5875,S,NO
16,0,3,male,2.0,4,1,29.1250,Q,NO


In [29]:
print(train_df["ticket_type"].unique())

['NO' 'SOC' 'A5' 'PC' 'CA' 'A4' 'WC' 'SOPP' 'SCA4' 'STONO' 'FCC' 'SP'
 'SCPARIS' 'SCAH' 'SWPP' 'STONO2' 'C' 'SCParis' 'PPP' 'SC' 'PP' 'WEP'
 'SOTONO2' 'AS' 'SOP']


In [30]:
train_df.dtypes

Survived         int64
Pclass           int64
Sex             object
Age            float64
SibSp            int64
Parch            int64
Fare           float64
Embarked        object
ticket_type     object
dtype: object

In [31]:
cols_str = ['Sex','Embarked','ticket_type']
for col in cols_str:
    train_df[col] = LabelEncoder().fit_transform(train_df[col])

In [32]:
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,ticket_type
483,1,3,0,63.0,0,0,9.5875,2,6
514,0,3,1,24.0,0,0,7.4958,2,6
870,0,3,1,26.0,0,0,7.8958,2,6
662,0,1,1,47.0,0,0,25.5875,2,6
16,0,3,1,2.0,4,1,29.1250,1,6


In [33]:
from sklearn.preprocessing import Normalizer
cols_num_scl = ['Age','Fare','SibSp']
nml = Normalizer(norm='l1')
train_df[cols_num_scl] = nml.fit_transform(train_df[cols_num_scl])

In [34]:
train_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,ticket_type
483,1,3,0,0.867918,0.000000,0,0.132082,2,6
514,0,3,1,0.762006,0.000000,0,0.237994,2,6
870,0,3,1,0.767057,0.000000,0,0.232943,2,6
662,0,1,1,0.647494,0.000000,0,0.352506,2,6
16,0,3,1,0.056940,0.113879,1,0.829181,1,6
...,...,...,...,...,...,...,...,...,...
42,0,3,1,0.785997,0.000000,0,0.214003,0,6
617,0,3,0,0.603248,0.023202,0,0.373550,2,1
571,1,1,0,0.497750,0.018783,0,0.483467,2,6
599,1,1,1,0.458247,0.009352,0,0.532401,0,7


In [35]:
#### Choosing feature columns and target columns for Supervised Learning

X_train = train_df.drop(columns=['Survived'])
y_train = train_df['Survived'] 

X_test = test_df.drop(columns=['Survived'])
y_test = test_df['Survived']



X_train, X_test,y_train, y_test = train_test_split(X_train,y_train, random_state=42, test_size=0.2, train_size=0.8)

mms = StandardScaler()
X_train = mms.fit_transform(X_train)
X_test = mms.fit_transform(X_test)

In [36]:
assert X_train.shape[0] == y_train.shape[0]
assert X_test.shape[0] == y_test.shape[0]

assert X_train.shape[1] == X_test.shape[1]
assert type(y_train) == type(y_test)

In [37]:
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)

print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

X_train: (569, 8)
y_train: (569,)
X_test: (143, 8)
y_test: (143,)


In [38]:
model_list = list()
for md in (1, 3, 5, 7, 9, None):
    for crit in ('gini', 'entropy','log_loss'):
        model_list.append(DecisionTreeClassifier(max_depth=md, criterion=crit,splitter='random'))
    
for k in (1, 3, 5, 7, 9):
    model_list.append(KNeighborsClassifier(n_neighbors=k))
    
print(model_list)
print(len(model_list))

[DecisionTreeClassifier(max_depth=1, splitter='random'), DecisionTreeClassifier(criterion='entropy', max_depth=1, splitter='random'), DecisionTreeClassifier(criterion='log_loss', max_depth=1, splitter='random'), DecisionTreeClassifier(max_depth=3, splitter='random'), DecisionTreeClassifier(criterion='entropy', max_depth=3, splitter='random'), DecisionTreeClassifier(criterion='log_loss', max_depth=3, splitter='random'), DecisionTreeClassifier(max_depth=5, splitter='random'), DecisionTreeClassifier(criterion='entropy', max_depth=5, splitter='random'), DecisionTreeClassifier(criterion='log_loss', max_depth=5, splitter='random'), DecisionTreeClassifier(max_depth=7, splitter='random'), DecisionTreeClassifier(criterion='entropy', max_depth=7, splitter='random'), DecisionTreeClassifier(criterion='log_loss', max_depth=7, splitter='random'), DecisionTreeClassifier(max_depth=9, splitter='random'), DecisionTreeClassifier(criterion='entropy', max_depth=9, splitter='random'), DecisionTreeClassifier

In [39]:
eval_dict = dict()
for model in model_list:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    eval_dict[str(model)] = f1_score(y_test, y_pred, average='micro')
pd.DataFrame.from_dict(eval_dict, orient='index', columns=['f1']).sort_values(by='f1', ascending=False)

,f1
"DecisionTreeClassifier(criterion='log_loss', max_depth=3, splitter='random')",0.832168
"DecisionTreeClassifier(max_depth=1, splitter='random')",0.811189
"DecisionTreeClassifier(criterion='log_loss', max_depth=1, splitter='random')",0.811189
"DecisionTreeClassifier(criterion='entropy', max_depth=1, splitter='random')",0.811189
"DecisionTreeClassifier(criterion='log_loss', max_depth=9, splitter='random')",0.797203
"DecisionTreeClassifier(criterion='log_loss', max_depth=7, splitter='random')",0.797203
"DecisionTreeClassifier(max_depth=7, splitter='random')",0.776224
"DecisionTreeClassifier(criterion='entropy', max_depth=3, splitter='random')",0.776224
KNeighborsClassifier(n_neighbors=9),0.769231
KNeighborsClassifier(n_neighbors=7),0.769231


In [40]:
f1 = f1_score(y_test, y_pred, average='micro')

print("F1 Score:", f1)

F1 Score: 0.7692307692307693


In [41]:
best_model = KNeighborsClassifier(n_neighbors=5) 
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

In [42]:
pd.DataFrame(confusion_matrix(y_test, y_pred))

,0,1
0,75,14
1,21,33


In [43]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.84      0.81        89
           1       0.70      0.61      0.65        54

    accuracy                           0.76       143
   macro avg       0.74      0.73      0.73       143
weighted avg       0.75      0.76      0.75       143



In [44]:
print('F1_score for micro is:',f1_score(y_test, y_pred, average='micro'))
print('F1_score for macro is:',f1_score(y_test, y_pred, average='macro'))
print('F1_score for weighted is:',f1_score(y_test, y_pred, average='weighted'))

F1_score for micro is: 0.7552447552447552
F1_score for macro is: 0.7321380786727321
F1_score for weighted is: 0.7513936424827514


In [45]:
##SVM
from sklearn import svm
#Create a svm Classifier
clf = svm.SVC(C=1.0,kernel='rbf',gamma='scale',decision_function_shape='ovr') 

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [46]:
print("Accuracy with f1:",metrics.f1_score(y_test, y_pred))
print("Accuracy with acc_score:",metrics.accuracy_score(y_test, y_pred))

Accuracy with f1: 0.7722772277227723
Accuracy with acc_score: 0.8391608391608392


In [47]:
# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred))

Precision: 0.8297872340425532
Recall: 0.7222222222222222


In [48]:
classifier_rf = RandomForestClassifier(n_estimators=88,max_depth=None,min_samples_split=2,min_samples_leaf=2,
                                       max_leaf_nodes=None,criterion='gini',max_features='sqrt')
classifier_rf.fit(X_train, y_train)

RandomForestClassifier(min_samples_leaf=2, n_estimators=88)

In [49]:
y_pred_rf = classifier_rf.predict(X_test)

In [50]:
y_pred_rf

array([1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], dtype=int64)

In [51]:
print("Accuracy for Random Forest:",metrics.accuracy_score(y_test, y_pred_rf))
print("f1_score for Random Forest:",metrics.f1_score(y_test, y_pred_rf))
print("Brier score for Random Forest:",metrics.brier_score_loss(y_test, y_pred_rf))
print("Balanced Accuracy for Forest:",metrics.balanced_accuracy_score(y_test, y_pred_rf))
print("Adjusted Mutal Info for Random Forest:",metrics.adjusted_mutual_info_score(y_test, y_pred_rf))
print("Average precision score for Random Forest:",metrics.average_precision_score(y_test, y_pred_rf))

Accuracy for Random Forest: 0.7552447552447552
f1_score for Random Forest: 0.6391752577319588
Brier score for Random Forest: 0.24475524475524477
Balanced Accuracy for Forest: 0.7196213066999584
Adjusted Mutal Info for Random Forest: 0.1644323927815176
Average precision score for Random Forest: 0.5747065165669816


In [52]:
print("Vasil Stamenkoski")

Vasil Stamenkoski
